In [27]:
import scipy
import numpy as np
import cv2
from keras.applications import inception_v3
from keras import backend as K
from keras.preprocessing import image

In [2]:
# Constants
DEMO_IMAGE_PATH = './test_images/flower_valley.jpg'

In [3]:
# Loop's hyperparameters
STEP = 0.01          # Value of the gradient ascent step
NUM_OCTAVE = 3       # Octave amount
OCTAVE_SCALE = 1.4   # Scale of each octave (look at the picture at the end of notebook)
ITERATIONS = 20      # Iterations amount on the each octave
MAX_LOSS = 10.       # Max loss that prevent ugly artifacts

In [4]:
K.set_learning_phase(0) #Set learning phase = 0 (test mode). Prevent model learning for safety reasons.

In [5]:
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

In [6]:
# Layers could be from mixed0 up to mixed10
# Check names with command model.summary()
layer_contribution = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5
}

In [7]:
# Loss calculation (L2_norm of layers outputs)
layer_dict = dict([(layer.name, layer) for layer in model.layers])
loss = K.variable(0.)

for layer_name in layer_contribution:
    coeff = layer_contribution[layer_name]
    # Get layer output
    activation = layer_dict[layer_name].output
    # Calculate L2 norm
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation)) / scaling  #test without -2:2 and so on

### Setting up gradient ascent

In [24]:
## Create K.function to fetch loss and gradients from model input

# Create tensor for result storing
dream = model.input 
# Fetch gradient and normalize it
grads = K.gradients(loss = loss, variables = dream)[0]
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)  #1e-7 - safety measure to avoid division by 0
# And now we provide link between current result and his gradients with losses
fetch_loss_and_grads = K.function([dream], [loss, grads])

# Gradient ascent's helper function
def eval_loss_and_grads(x):
    '''
    Extract loss and grads values from current input state
    Input:
        Current state of model.input
    Output:
        Current values of loss and input gradients
    '''
    # Pass input x (it will be model.input) through  K.function([model.input], [loss, grads])
    outs = fetch_loss_and_grads([x])
    # Get values
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values
    
# Main function
def gradient_ascent(x, iterations, step, max_loss=None, verbose=True):
    '''
    Performs the specified number of gradient ascent steps.
    '''
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if verbose:
            print('Loss value at {} step: {:.3f}'.format(i, loss_value))
        if max_loss is not None and loss_value > max_loss:
            print('Current loss = {:.3f} exceeded max_loss = {:.3f}, ascent was finished'.format(loss_value, max_loss))
            break
        x += step * grad_values
    return x

### Deep Dream main loop

In [17]:
# Base helper functions

def preprocess_img(img_path):
    '''
    Convert raw image to the InceptionV3 input format
    Input:
        Path to file, str
    Output:
        Numpy arrray with shape [1, img_height, img_width, 3]
    '''
    img = image.load_img(img_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def resize_img(img, size):
    '''
    Resize input image by zooming it
    Input:
        Numpy matrix with shape [1, img_height, img_width, 3]
    Output:
        Numpy matrix with shape [1, zoomed_img_height, zoomed_img_width, 3]
    '''
    img = np.copy(img)
    factor = (1, float(size[0])/img.shape[1], float(size[1])/img.shape[2], 1)
    return scipy.ndimage.zoom(img, factor, order=1) #order - spline interpolation order. Could be in range(0, 5)

def deprocess_img(x):
    '''
    Convert InceptionV3's result input after gradient ascent to the np.array image
    Input:
        Numpy arrray with shape [1, img_height, img_width, 3]
    Output:
        Numpy arrray with shape [img_height, img_width, 3]
    '''
    if K.image_data_format() == 'channel_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape(x.shape[1], x.shape[2], 3)  
    # Undo InceptionV3 preprocess input
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x
    

def save_img(img, file_name):
    '''
    Save image into png format
    Input:
        Numpy arrray with shape [1, img_height, img_width, 3]
    Output:
        None
    '''
    pil_img = deprocess_img(np.copy(img))
    cv2.imwrite(file_name, pil_img)

![alt text](./notebook_images/img1.png)

In [18]:
# # Loop's hyperparameters
# STEP = 0.01          # Value of the gradient ascent step
# NUM_OCTAVE = 3       # Octave amount
# OCTAVE_SCALE = 1.4   # Scale of each octave (look at the picture at the end of notebook)
# ITERATIONS = 20      # Iterations amount on the each octave
# MAX_LOSS = 10.       # Max loss that prevent ugly artifacts

In [28]:
img = preprocess_img(DEMO_IMAGE_PATH)

# Create list of shapes correspond with octave scales 
original_shape = img.shape[1:3]
octave_shapes = [original_shape]
for i in range(NUM_OCTAVE):
    scaled_shape = tuple([int(dim/(OCTAVE_SCALE ** i)) for dim in original_shape])
    octave_shapes.append(scaled_shape)
octave_shapes = octave_shapes[::-1]


orginal_img = np.copy(img)

# Initialize shrunck image by the smallest image
shrunck_original_img = resize_img(img, octave_shapes[0]) 

for shape in octave_shapes:
    print('Processing image shape: ', shape)
    # Image gradient ascenting 
    img = resize_img(img, shape)
    img = gradient_ascent(img, ITERATIONS, STEP, MAX_LOSS)
    # Lost detail computation
    upscaled_shrunck_original_img = resize_img(shrunck_original_img, shape)
    same_original_size = resize_img(orginal_img, shape)
    lost_detail = same_original_size - upscaled_shrunck_original_img
    # Impute details
    img += lost_detail
    save_img(img, './notebook_images/scale_{}.png'.format(shape))
    
    shrunck_original_img = resize_img(orginal_img, shape)
    
save_img(img, './notebook_images/result.png') 
print('Process finished, result was saved in the project root folder')

Processing image shape:  (271, 408)
Loss value at 0 step: 2.4181
Loss value at 1 step: 3.1589
Loss value at 2 step: 4.0558
Loss value at 3 step: 4.9460
Loss value at 4 step: 5.8067
Loss value at 5 step: 6.6607
Loss value at 6 step: 7.5126
Loss value at 7 step: 8.3378
Loss value at 8 step: 9.1201
Loss value at 9 step: 9.9231
Loss value at 10 step: 10.6791
Current loss = 10.679125 exceeded max_loss = 10.000000, ascent was finished
Processing image shape:  (380, 571)
Loss value at 0 step: 3.3760
Loss value at 1 step: 4.6584
Loss value at 2 step: 5.8179
Loss value at 3 step: 6.8233
Loss value at 4 step: 7.8227
Loss value at 5 step: 8.7256
Loss value at 6 step: 9.6203
Loss value at 7 step: 10.4525
Current loss = 10.452514 exceeded max_loss = 10.000000, ascent was finished
Processing image shape:  (533, 800)
Loss value at 0 step: 3.1636
Loss value at 1 step: 4.3904
Loss value at 2 step: 5.4599
Loss value at 3 step: 6.4159
Loss value at 4 step: 7.3452
Loss value at 5 step: 8.2259
Loss value a